# Textagon System Demonstration

This Colab file demonstrates the functionality of the Textagon software.
Because of Google Colab resource constraints and RAM limitations, in certain spots, we rely on pre-processed portions of the Textagon pipeline.
In those places, we include the command that was run offline to generate the preprocessed component.



##### Arguments:
1. inputFileFullPath: Path to the input text file
2. outputFileName: Name for the output file
3. inputLimit: Limit on the number of input files (0 means no limit)
4. maxFeatures: Maximum number of features to extract (0 means no limit)
5. maxNgram: Maximum n-gram size (1 for unigrams, 2 for bigrams, etc.)
6. maxCores: Number of CPU cores to use
7. lexiconFileFullPath: Path to the lexicon file (zip or folder containing .txt files)
8. vader: Whether to use VADER sentiment analysis (1 for true, 0 for false)
9. wnaReturnLevel: WNA return level
10. buildVectors: Build vectors setting (custom value, e.g., 'bB')
11. index: Whether to index the output (1 for true, 0 for false)
12. removeZeroVariance: Remove zero variance features (1 for true, 0 for false)
13. combineFeatures: Combine features (1 for true, 0 for false)
14. minDF: Minimum document frequency (integer or float value)
15. removeDupColumns: Remove duplicate columns (1 for true, 0 for false)
16. useSpellChecker: Use spell checker (1 for true, 0 for false)
17. provideMisspellingDetailed: Provide detailed misspelling information (1 for true, 0 for false)
18. additionalCols: Include additional columns in the output (1 for true, 0 for false)
19. writeRepresentations: Write representations to the output (1 for true, 0 for false)
20. exclusionsFileFullPath: Path to the exclusions file
21. runType: Type of run (e.g., 'full', 'partial')

```python
mkdir -p content/output
python process-text.py \
    ./sample_data_1/textagon_sample/raw.txt \
    ./sample_data_1/textagon_sample \
    0 0 1 4 external/lexicons/Lexicons_v5.zip 1 5 bB 0 1 0 3 1 1 0 1 1 upload/exclusions.txt full


#### 5. Compare BERT only and BERT with textagon

After you have generated the parallel representations, you can plug them into a BERT classifier pipeline.

**Note**: The below section of code takes several hours to run.
The results are included in the manuscript.

In [5]:
import os
#from datasets import load_dataset
import os
import pandas as pd
import csv
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tabulate import tabulate
import matplotlib.pyplot as plt
import six
import re
import nltk
from nltk.corpus import words
#from huggingface_hub import HfApi

import os
import pandas as pd
import csv
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tabulate import tabulate
import matplotlib.pyplot as plt
import six
import re
import nltk
from nltk.corpus import words

import os
import pandas as pd
import csv
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tabulate import tabulate
import matplotlib.pyplot as plt
import six
import re
import nltk
from nltk.corpus import words
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
import re

import os, warnings, torch, nltk, csv, logging
from keras import *
from gensim.models import *
from sklearn.metrics import *
import matplotlib.pyplot as plt
from keras.preprocessing import *
from numpy.random import seed
seed(1)


2025-03-25 08:57:55.407020: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 08:57:55.417481: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 08:57:55.427097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742907475.441392  990753 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742907475.445500  990753 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

In [7]:

### For wordcnn ###
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score, accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
if not nltk.data.find('tokenizers/punkt'): nltk.download('punkt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # device

### For bert ###
from transformers import BertTokenizer
from sklearn.metrics import roc_auc_score, accuracy_score
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

# from utils import split_into_train_valid_test, save_plot_and_metric, load_or_make
# from utils import get_prediction_E2E, data_wordcnn_E2E
# from utils import set_bert_model, MLP

import string
import pandas as pd
from itertools import combinations
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score

import torch
import torch.nn as nn
import collections
from collections import Counter
from itertools import chain
from torch.nn.functional import softmax
from torch.utils.data import Dataset, DataLoader
from gensim.models import Word2Vec
import numpy as np

from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertTokenizer, BertModel, BertForSequenceClassification


/afs/crc.nd.edu/user/j/jlalor1/.conda/envs/textagon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

def clean_text(text):
    return ''.join([ch.lower() for ch in text if ch not in string.punctuation])

class Dataset_bert(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, max_length=self.max_length, padding='max_length', return_tensors='pt')
        return {'input_ids': encoding['input_ids'].squeeze(),
                'attention_mask': encoding['attention_mask'].squeeze(),
                'labels': torch.tensor(label).long()}

def process_bert_inputs(texts, tokenizer, max_length):
    encodings = [tokenizer(text, truncation=True, max_length=max_length, padding='max_length') for text in texts]
    input_ids = torch.tensor([enc['input_ids'] for enc in encodings], dtype=torch.long)
    attention_masks = torch.tensor([enc['attention_mask'] for enc in encodings], dtype=torch.long)
    return input_ids, attention_masks

def fbeta_score(y_true, y_pred, beta):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return (1 + beta**2) * precision * recall / (beta**2 * precision + recall)

def process_data_for_transformer(data_path, test_size, tokenizer, max_length, batch_size):
    data = open(data_path, 'r', errors='replace').readlines()
    labels = [int(row.strip().split('\t')[0]) for row in data]
    texts = [row.strip().split('\t')[1] for row in data]
    train_text, test_text, training_label, testing_label = train_test_split(texts, labels, test_size=test_size, random_state=42)
    test_text, void_text, testing_label, void_label = train_test_split(test_text, testing_label, test_size=0.5, random_state=42)
    train_dataset_bt = Dataset_bert(train_text, training_label, tokenizer, max_length)
    val_dataset_bt = Dataset_bert(test_text, testing_label, tokenizer, max_length)
    train_loader_bt = DataLoader(train_dataset_bt, batch_size=batch_size, shuffle=False)
    val_loader_bt = DataLoader(val_dataset_bt, batch_size=batch_size, shuffle=False)
    return train_loader_bt, val_loader_bt

class TransformerClassifier(nn.Module):
    def __init__(self, feature_extractor, cls_emb_size, num_classes):
        super().__init__()
        self.transformer = feature_extractor
        self.classifier = nn.Linear(cls_emb_size, num_classes)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        # using pooled_output corresponding to the [CLS] token
        if isinstance(outputs, tuple):
            pooled_output = outputs[1]  # assuming output_hidden_states and output_attentions are False
        else:
            pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        return logits


def get_full_combos(lst, max_length=None):
    """
    Generate combinations of elements from a list, ensuring the first element is always included.

    Args:
    - lst (list): A list of elements.
    - max_length (int, optional): The maximum allowed length for the resulting combinations.
                                  If not provided, combinations of all lengths are allowed.

    Returns:
    - list: A list of combinations. Each combination is a list of elements.

    Example:
        lst = [1, 2, 3, 4]
        print(get_full_combos(lst))
        Output:
        [[1],
         [2],
         [3],
         [4],
         [1, 2],
         [1, 3],
         [1, 4],
         [1, 2, 3],
         [1, 2, 4],
         [1, 3, 4],
         [1, 2, 3, 4]]

        print(get_full_combos(lst, max_length=2))
        Output:
        [[1],
         [2],
         [3],
         [4],
         [1, 2],
         [1, 3],
         [1, 4]]
    """
    result = []
    for i in range(1, len(lst) + 1):
        for combo in combinations(lst, i):
            # If max_length is provided, ensure the combo length does not exceed it
            if max_length and len(combo) > max_length:
                continue
            result.append(list(combo))
    return result

def get_rep_linear_combos(lst):
    result = []
    for i in range(1, len(lst) + 1):
        result.append(list(lst[:i]))
    return result

def process_texts_E2E(texts, word_to_ix, max_sentence_length):
    processed = [simple_tokenize(text) for text in texts]
    processed = [[word_to_ix[word] if word in word_to_ix else word_to_ix['<UNK>'] for word in text] for text in processed]
    processed = [text[:max_sentence_length] + [word_to_ix['<PAD>']]*(max_sentence_length - len(text)) for text in processed]
    return torch.tensor(processed, dtype=torch.long)

def data_wordcnn_E2E(trainText, validationText, testText, trainingLabels, validationLabels, testLabels, word_to_ix, batch_size, MAX_SENTENCE_LENGTH):

    train_data = process_texts_E2E(trainText, word_to_ix, MAX_SENTENCE_LENGTH)
    train_labels = torch.tensor(trainingLabels, dtype=torch.long)

    validation_data = process_texts_E2E(validationText, word_to_ix, MAX_SENTENCE_LENGTH)
    validation_labels = torch.tensor(validationLabels, dtype=torch.long)

    test_data = process_texts_E2E(testText, word_to_ix, MAX_SENTENCE_LENGTH)
    test_labels = torch.tensor(testLabels, dtype=torch.long)

    train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    validation_dataset = torch.utils.data.TensorDataset(validation_data, validation_labels)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, validation_loader, test_loader


def create_3d_loader(loader_list, shuffle=False):
    data_list, label_list = [], []
    for data_and_labels in zip(*loader_list):
        data = torch.cat([item[0].unsqueeze(1) for item in data_and_labels], dim=1)
        label = data_and_labels[0][1]  # assuming the labels are the same across all loaders
        data_list.append(data)
        label_list.append(label)
    new_loader = list(zip(data_list, label_list))

    if shuffle:
        random.shuffle(new_loader)

    return new_loader

class CNN2D_E2E_AE(nn.Module):
    def __init__(self, vocab_size, num_filters, num_class, num_channels, width, auto_encoder_path):
        super(CNN2D_E2E_AE, self).__init__()

        self.embed = nn.Embedding(vocab_size, width)

        self.autoencoder = Autoencoder(vocab_size, width)
        self.autoencoder.load_state_dict(torch.load(f'{auto_encoder_path}/autoencoder.pth'))  # Load trained weights
        self.autoencoder = self.autoencoder.encoder

        conv_dim = int(width * 2/3)
        self.conv1_1 = nn.Sequential(
            nn.Conv2d(num_channels, num_filters, kernel_size=(1, conv_dim // 6), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, conv_dim - (conv_dim // 6) + 1))
        ).cuda()
        self.conv1_2 = nn.Sequential(
            nn.Conv2d(num_channels, num_filters, kernel_size=(2, conv_dim // 4), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, conv_dim - (conv_dim // 4) + 1))
        ).cuda()
        self.conv1_3 = nn.Sequential(
            nn.Conv2d(num_channels, num_filters, kernel_size=(3, conv_dim // 3), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, conv_dim - (conv_dim // 3) + 1))
        ).cuda()
        self.fc = nn.Linear(num_filters, num_class)
        self.adaptive_pool = nn.AdaptiveMaxPool2d((None, 1))

    def forward(self, x, output_last_layer=False):
        # print('0:', x.shape) # torch.Size([8, 1, 500])
        x = x.long() # Make sure x is of type LongTensor for Embedding Layer
        x = self.embed(x)
        # print('1:', x.shape) # torch.Size([8, 1, 500, 500])
        x = self.autoencoder(x)
        # print('2:', x.shape) # torch.Size([8, 1, 500, 333])
        # x = x.unsqueeze(1) # [batch_size, 1, num_models, seq_length, emb_dim]
        c1 = self.conv1_1(x).squeeze(-1)
        # print('c1:', c1.shape)
        c2 = self.conv1_2(x).squeeze(-1)
        c3 = self.conv1_3(x).squeeze(-1)
        # print("3:", c1.shape, c2.shape, c3.shape) # torch.Size([8, 600, 500]) torch.Size([8, 600, 499]) torch.Size([8, 600, 498])
        c = torch.cat([c1, c2, c3], dim = -1)
        # print("4:", c.shape) # torch.Size([8, 600, 1497])
        c = self.adaptive_pool(c).squeeze(-1)
        # print("5:", c.shape) # torch.Size([8, 600])
        c = c.view(c.size(0), -1)  # Flatten tensor while preserving the batch size
        # print("6:", c.shape)
        if output_last_layer:
            return c
        else:
            logit = self.fc(c)
            return logit

class Autoencoder(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(emb_dim, int(emb_dim * (4/5))),
            nn.ReLU(True),
            nn.Linear(int(emb_dim * (4/5)), int(emb_dim * (3/4))),
            nn.ReLU(True),
            nn.Linear(int(emb_dim * (3/4)), int(emb_dim * (2/3))),
            nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(int(emb_dim * (2/3)), int(emb_dim * (3/4))),
            nn.ReLU(True),
            nn.Linear(int(emb_dim * (3/4)), int(emb_dim * (4/5))),
            nn.ReLU(True),
            nn.Linear(int(emb_dim * (4/5)), emb_dim),
            nn.ReLU(True),
        )

    def forward(self, x):
        x_encoded = self.encoder(x)
        x_decoded = self.decoder(x_encoded)
        return x_decoded

def fbeta_score(y_true, y_pred, beta):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return (1 + beta**2) * precision * recall / (beta**2 * precision + recall)

def check_loader_shapes(loader_list):
    shape_dict = {}
    for i, dataloader in enumerate(loader_list):
        for j, (input, label) in enumerate(dataloader):
            if j not in shape_dict:
                shape_dict[j] = input.shape
            elif shape_dict[j] != input.shape:
                return False
    return True

def simple_tokenize(text):
    return text.split()



##### Run BERT only

In [9]:

original_data_path = f'./sample_data'

"""Load parameters"""
num_classes = 2  # number of output classes
###### Bert ######
lr_bert = 1e-5
batch_size_bt = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length_bert = 128
epochs = 5
epochs = 1
test_data = 0.3

train_loader_transformer, test_loader_transformer =  process_data_for_transformer(
    data_path = f'{original_data_path}/distress_raw.txt',
    test_size = test_data,
    tokenizer=tokenizer,
    max_length = max_length_bert,
    batch_size=batch_size_bt
)

config_bert = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_classes)
model_bert = TransformerClassifier(
        feature_extractor = BertModel.from_pretrained('bert-base-uncased'),
        cls_emb_size = config_bert.hidden_size,
        num_classes = num_classes).to(device)
optimizer_bert = torch.optim.AdamW(model_bert.parameters(), lr=lr_bert)

best_auc = 0.0
best_preds, best_labels = None, None
for epoch in range(epochs):
    # log_file = open(f'{save_batch_path}/log.txt', 'a')
    model_bert.train()
    train_loss = 0
    train_preds, train_targets = [], []
    for inputs_trans in train_loader_transformer:
        optimizer_bert.zero_grad()
        labels = inputs_trans['labels'].to(device)
        inputs_trans, attention_mask_trans = inputs_trans['input_ids'].to(device), inputs_trans['attention_mask'].to(device)
        outputs_trans = model_bert(inputs_trans, attention_mask=attention_mask_trans)
        loss = F.cross_entropy(outputs_trans, labels)
        loss.backward()
        optimizer_bert.step()
        train_loss += loss.item()
        preds = torch.argmax(outputs_trans, dim=-1)
        train_preds.extend(preds.cpu().numpy())
        train_targets.extend(labels.cpu().numpy())

    train_loss /= len(train_loader_transformer)  # average loss over the epoch
    train_acc = (np.array(train_preds) == np.array(train_targets)).mean()  # accuracy
    train_auc = roc_auc_score(train_targets, train_preds)  # AUC

    train_recall = recall_score(train_targets, train_preds)  # recall
    train_f1 = f1_score(train_targets, train_preds)  # F1
    train_f2 = fbeta_score(train_targets, train_preds, beta=2)  # F2

    model_bert.eval()
    test_loss = 0
    test_preds, test_targets = [], []
    with torch.no_grad():
        for inputs_trans in test_loader_transformer:
            labels = inputs_trans['labels'].to(device)
            inputs_trans, attention_mask_trans = inputs_trans['input_ids'].to(device), inputs_trans['attention_mask'].to(device)
            outputs_trans = model_bert(inputs_trans, attention_mask=attention_mask_trans)
            loss = F.cross_entropy(outputs_trans, labels)
            test_loss += loss.item()
            preds = torch.argmax(outputs_trans, dim=-1)
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    test_loss /= len(test_loader_transformer)  # average loss over the epoch
    test_acc = (np.array(test_preds) == np.array(test_targets)).mean()  # accuracy
    test_auc = roc_auc_score(test_targets, test_preds)  # AUC
    test_recall = recall_score(test_targets, test_preds)  # recall
    test_f1 = f1_score(test_targets, test_preds)  # F1
    test_f2 = fbeta_score(test_targets, test_preds, beta=2)  # F2

stats = f'Stats: \n' \
            f'Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, F2: {train_f2:.4f}\n' \
            f'Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, F2: {test_f2:.4f}\n\n'
print(stats)




Stats: 
Train Loss: 0.6879, Acc: 0.5568, AUC: 0.5567, Recall: 0.5533, F1: 0.5467, F2: 0.5506
Test Loss: 0.6827, Acc: 0.5878, AUC: 0.5872, Recall: 0.4029, F1: 0.4934, F2: 0.4348




##### Run BERT with textagon

In [ ]:
# JL
### MAYBE SKIP THIS CELL??? ###

"""load GBS weights from excel file"""

textagon_path = './textagon.xlsx'
df = pd.read_excel(textagon_path)
df = df.drop(df.columns[0], axis=1)

features = df.iloc[0, 1:].tolist()
result_dict = {}

drop_features = []

# Iterating from the second row onwards
for index, row in df.iloc[1:].iterrows():
    dataset_name = row[0]
    paired = list(zip(features, row[1:]))  # Pair each feature with its value

    # Filter out the undesired features
    # paired = [pair for pair in paired if pair[0] not in drop_features]

    paired_sorted = sorted(paired, key=lambda x: x[1], reverse=True)  # Sort pairs based on the value
    sorted_features, sorted_values = zip(*paired_sorted)  # Separate the pairs back into lists
    result_dict[dataset_name] = (list(sorted_features), list(sorted_values))  # Save into the dictionary

    # Move "Word" to the front
    if "Word" in sorted_features:
        word_index = sorted_features.index("Word")
        sorted_features = ["Word"] + [f for i, f in enumerate(sorted_features) if i != word_index]
        word_value = sorted_values[word_index]
        sorted_values = [word_value] + [v for i, v in enumerate(sorted_values) if i != word_index]

    result_dict[dataset_name] = (list(sorted_features), list(sorted_values))  # Save into the dictionary

# print(result_dict['Anxiety'][0])
# print(result_dict['Anxiety'][-1])


In [ ]:
'''This is pick the accumulatively lowest cosine similarity between WORD representations and others'''

representations_folder = f'./output/distress_representations'
representation_list = os.listdir(representations_folder)
for rep_temp in representation_list:
    ## Rename the file
    new_name = rep_temp.split('_')[-1]
    os.rename(f'{representations_folder}/{rep_temp}', f'{representations_folder}/{new_name}')


###### helper functions ######
def simple_tokenize(text):
    return text.split()

def process_texts_E2E(texts, word_to_ix, max_sentence_length):
    processed = [simple_tokenize(text) for text in texts]
    processed = [[word_to_ix[word] if word in word_to_ix else word_to_ix['<UNK>'] for word in text] for text in processed]
    processed = [text + [word_to_ix['<PAD>']] * (max_sentence_length - len(text)) for text in processed]
    return torch.tensor(processed, dtype=torch.long)

###### Get the expressive scores ######
textagon_path = './textagon.xlsx'
df = pd.read_excel(textagon_path)
df = df.drop(df.columns[0], axis=1)
features = df.iloc[0, 1:].tolist()
result_dict = {}

for index, row in df.iloc[1:].iterrows():
    dataset_name = row[0]
    paired = list(zip(features, row[1:]))
    paired_sorted = sorted(paired, key=lambda x: x[1], reverse=True)
    sorted_features, sorted_values = zip(*paired_sorted)
    result_dict[dataset_name] = (list(sorted_features), list(sorted_values))

###### Get embeddings for datasets ######
# # trans_model = 'final_v2_17_distilbert'
# datasets = os.listdir(data_folder)
# datasets = [d for d in datasets if '.' not in d]
representations = os.listdir(representations_folder)
high_path = './data/bs'
dataset_dict = dict()


master_vocab = set()
for rep in representations:
    rep = rep.split('_')[-1]
    if rep == 'Boundaries.txt': continue
    texts = open(f'{representations_folder}/{rep}', 'r', errors='replace').readlines()
    texts_tokened = [simple_tokenize(text) for text in texts]
    for sentence in texts_tokened:
        master_vocab.update(sentence)

original_tokens = sorted([word for word in master_vocab if not word.isupper()], key=lambda x: (not x.isalpha(), x))
replaced_tokens = sorted([word for word in master_vocab if word.isupper()])
word_to_ix = {'<UNK>': 0, '<PAD>': 1}
for i, word in enumerate(original_tokens + replaced_tokens, start=2):
    word_to_ix[word] = i

length_list = []
for rep in representations:
    rep = rep.split('_')[-1]
    if rep == 'Boundaries.txt': continue
    texts = open(f'{representations_folder}/{rep}', 'r', errors='replace').readlines()
    texts_tokened = [simple_tokenize(text) for text in texts]
    max_sentence_length = max(len(sentence) for sentence in texts_tokened)
    length_list.append(max_sentence_length)
length = max(length_list)

rep_dict = dict()
for rep in representations:
    rep = rep.split('_')[-1]
    if rep == 'Boundaries.txt': continue
    texts = open(f'{representations_folder}/{rep}', 'r', errors='replace').readlines()
    rep_dict[rep.split('.')[0]] = process_texts_E2E(texts=texts, word_to_ix=word_to_ix, max_sentence_length=length)
dataset_dict['distress'] = (rep_dict, word_to_ix, length)


def cosine_similarity_v0(matrix1, matrix2):
    flattened1 = matrix1.float().flatten()
    flattened2 = matrix2.float().flatten()
    return F.cosine_similarity(flattened1.unsqueeze(0), flattened2.unsqueeze(0))


def get_highest_expressive_power_rep(dataset, rank=1):
    all_reps = result_dict[dataset][0]
    expressive_values = result_dict[dataset][1]

    # Filter out 'Word'
    reps_filtered = [rep for rep in all_reps if rep != 'Word']
    values_filtered = [expressive_values[all_reps.index(rep)] for rep in reps_filtered]

    sorted_indices = sorted(range(len(values_filtered)), key=lambda k: values_filtered[k], reverse=True)
    return reps_filtered[sorted_indices[rank-1]]


def get_lowest_cosine_similarity_rep(dataset, rank=1):
    word_rep = dataset_dict[dataset][0]['Word']
    top_7_reps = result_dict[dataset][0][:7]
    similarities = {}
    for rep in top_7_reps:
        if rep != 'Word':
            similarities[rep] = cosine_similarity_v0(word_rep, dataset_dict[dataset][0][rep]).item()
    sorted_reps = sorted(similarities, key=similarities.get)
    return sorted_reps[rank-1]

def get_best_auc_for_epoch(selected_epoch, path = './data/bs'):
    # dataset = 'distress'
    datasets = ['distress']
    optimal_solutions = dict()

    for dataset in datasets:
        # rep_dict = dict()
        # for rep_combination in os.listdir(f'{path}/{dataset}'):
        #     rep_dict[frozenset(rep_combination.split('_'))] = rep_combination

        # print(f"\nFor dataset {dataset}:")

        comb_names = []
        temp_names = []

        # 1. "Word"
        comb_names.append('Word')

        # 2. Highest expressive power representation (exp 1)
        exp1 = get_highest_expressive_power_rep(dataset, 1)
        comb_names.append(f"{exp1}")

        # 3. Second highest expressive power representation (exp 2)
        exp2 = get_highest_expressive_power_rep(dataset, 2)
        comb_names.append(f"{exp2}")

        # 3-1. Third highest expressive power representation (exp 2)
        exp3 = get_highest_expressive_power_rep(dataset, 3)
        comb_names.append(f"{exp3}")

        # 4. & 5. Lowest and Second Lowest cosine similarity
        cos1 = get_lowest_cosine_similarity_rep(dataset, 1)
        comb_names.append(f"{cos1}")

        cos2 = get_lowest_cosine_similarity_rep(dataset, 2)
        comb_names.append(f"{cos2}")

        for cn in comb_names:
            temp_names.append([cn])

        # Combined representations using dictionary
        combinations = [
            (['Word', exp1], 'word + exp1'),
            (['Word', exp1, cos1], 'word + exp1 + cos1'),
            (['Word', exp2, cos1], 'word + exp2 + cos1'),
            (['Word', exp1, exp2], 'word + exp1 + exp2'),
            (['Word', exp1, exp2, exp3], 'word + exp1 + exp2 + exp3'),
            (['Word', exp1, cos1, exp2], 'word + exp1 + cos1 + exp2'),
            (['Word', exp1, exp2, cos2], 'word + exp1 + cos2 + exp2')
        ]

        for combo, mark_name in combinations:
            # file_name = rep_dict[combo]
            # t_list = file_name.split('_')
            # t_list.sort()
            temp_names.append(combo)

        optimal_solutions[dataset] = temp_names
    return optimal_solutions

### Concatenation
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )
    def forward(self, x):
        return self.layers(x)


# Example usage:
optimal_space_dict = get_best_auc_for_epoch(4)


In [ ]:
"""Load data"""
accumulation_steps = 4

def process_data_for_transformer(data_path, test_size, val_size, tokenizer, max_length, batch_size):
    data = open(data_path, 'r', errors='replace').readlines()
    labels = [int(row.strip().split('\t')[0]) for row in data]
    texts = [row.strip().split('\t')[1] for row in data]
    temp_text, test_text, temp_label, testing_label = train_test_split(texts, labels, test_size=test_size, random_state=42)
    train_text, val_text, training_label, val_label = train_test_split(temp_text, temp_label, test_size=val_size, random_state=42)

    train_dataset_bt = Dataset_bert(train_text, training_label, tokenizer, max_length)
    val_dataset_bt = Dataset_bert(val_text, val_label, tokenizer, max_length)
    test_dataset_bt = Dataset_bert(test_text, testing_label, tokenizer, max_length)

    train_loader_bt = DataLoader(train_dataset_bt, batch_size=batch_size, shuffle=False)
    val_dataset_bt = DataLoader(val_dataset_bt, batch_size=batch_size, shuffle=False)
    test_loader_bt = DataLoader(test_dataset_bt, batch_size=batch_size, shuffle=False)
    return train_loader_bt, val_dataset_bt, test_loader_bt

# for bench in bs:

bench = 'distress'
original_data_path = f'./sample_data_1/textagon_sample/'
reps_folder_path = f'./sample_data_1/textagon_sample/distress_representations'
sorted_reps = result_dict[bench][0]
textagon_weights = result_dict[bench][1]

### Get a word file with labels, combine word and raw
word_file = open(f'{reps_folder_path}/Word.txt', 'r', errors='replace').readlines()
raw_file = open(f'{original_data_path}/raw.txt', 'r', errors='replace').readlines()
new_word_file = []
for i, line in enumerate(word_file):
    sample = word_file[i]
    label = raw_file[i].strip().split('\t')[0]
    new_word_file.append(f'{label}\t{sample}')
with open(f'{original_data_path}/Word.txt', 'w') as f:
    for item in new_word_file:
        f.write("%s" % item)


print(f'================={bench}=================')

"""Load parameters"""
###### WordCNN ######
### End-to-end Embedding ###
emb_dim = 256  # embedding dimension
num_filters = 400  # number of filters in conv layers
num_classes = 2  # number of output classes
lr_wordcnn = 1e-3
lr_autoencoder = 1e-3
batch_size_wc = 8
min_frq_threshold = 2  # threshold for filtering rare words
###### autoencoder
encoding_dim = 100 # Specify the auto encoding dim
### Pre-train (Word to Vec) ###
window_size = 5  # context window size
min_frq_threshold = 2  # threshold for filtering rare words
EMBEDDING_DIMENSION = emb_dim
isSkipgram = 1
iterations = 30
###### Bert ######
max_length = 128
batch_size_bt = 8
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length_bert = 128
lr_bert = 1e-5
###### Concatenation part ######
lr_mlp = 1e-3
###### train and eval
E2E = True # End-to-end embedding or word-to-vector embedding
epochs = 10
test_data = 0.2
val_data_size = 1/32
number_reps_keep = 7
reps_train = 3

train_loader_transformer, validation_loader_transformer, test_loader_transformer =  process_data_for_transformer(
    data_path = f'{original_data_path}/Word.txt',
    test_size = test_data,
    val_size = val_data_size,
    tokenizer=tokenizer,
    max_length = max_length_bert,
    batch_size=batch_size_bt
)

# Create a master vocabulary for the dataset
master_vocab = set()
for rep in sorted_reps:
    if rep == 'Boundaries.txt': continue
    texts = open(f'{reps_folder_path}/{rep}.txt', 'r', errors='replace').readlines()
    texts_tokened = [simple_tokenize(text) for text in texts]
    for sentence in texts_tokened:
        master_vocab.update(sentence)
# Sort the master vocabulary and create word_to_idx mapping
original_tokens = sorted([word for word in master_vocab if not word.isupper()], key=lambda x: (not x.isalpha(), x))
replaced_tokens = sorted([word for word in master_vocab if word.isupper()])
word_to_ix = {'<UNK>': 0, '<PAD>': 1}
for i, word in enumerate(original_tokens + replaced_tokens, start=2):
    word_to_ix[word] = i
length_list = []
for rep in sorted_reps:
    if rep == 'Boundaries.txt': continue
    texts = open(f'{reps_folder_path}/{rep}.txt', 'r', errors='replace').readlines()
    texts_tokened = [simple_tokenize(text) for text in texts]
    max_sentence_length = max(len(sentence) for sentence in texts_tokened)
    length_list.append(max_sentence_length)
length = max(length_list)

rep_dict = dict()
final_reps = []
# print(sorted_reps)
for rep in sorted_reps:
    if len(final_reps) == number_reps_keep: continue
    texts = open(f'{reps_folder_path}/{rep}.txt', 'r', errors='replace').readlines()
    labels = open(f'{original_data_path}/Word.txt', 'r', errors='replace').readlines()
    texts = [row.strip() for row in texts]
    labels = [int(row.strip().split('\t')[0]) for row in labels]
    temp_text, test_text, temp_label, testing_label = train_test_split(texts, labels, test_size=test_data, random_state=42)
    train_text, val_text, training_label, val_label = train_test_split(temp_text, temp_label, test_size=val_data_size, random_state=42)

    train_tokens = [simple_tokenize(text) for text in train_text]
    val_tokens = [simple_tokenize(text) for text in val_text]
    test_tokens = [simple_tokenize(text) for text in test_text]

    final_reps.append(rep)
    train_loader, val_loader, test_loader = data_wordcnn_E2E(train_text, val_text, test_text, training_label, val_label, testing_label, word_to_ix, batch_size_wc, length)
    rep_dict[str(rep)] = (word_to_ix, train_loader, val_loader, test_loader)

optimal_names = optimal_space_dict[bench]
best_auc = 0.0
for rep_list in optimal_names:
    rep_name = '_'.join([row for row in rep_list])
    rep_length = len(rep_list)
    word_to_ix_list = [rep_dict[item][0] for item in rep_list]
    train_loader_list = [rep_dict[item][1] for item in rep_list]
    val_loader_list = [rep_dict[item][2] for item in rep_list]
    test_loader_list = [rep_dict[item][3] for item in rep_list]

    if check_loader_shapes(train_loader_list) == False or check_loader_shapes(test_loader_list) == False:
        print('check,', rep_name)
        continue

    train_loader_3D = create_3d_loader(train_loader_list)
    val_loader_3D = create_3d_loader(val_loader_list)
    test_loader_3D = create_3d_loader(test_loader_list)
    batch_size, num_of_reps, max_sentence_length = list(train_loader_3D[0][0].shape)

    vocab_size = len(word_to_ix)
    max_sentence_length = 500

    autoencoder = Autoencoder(vocab_size, max_sentence_length).to(device)
    autoencoder_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
    embedding_layer = nn.Embedding(vocab_size, max_sentence_length).to(device)

    criterion = nn.MSELoss()
    # counter = 0
    for epoch in range(10):
        for inputs, _ in train_loader_3D:  # Ignore labels
            inputs = inputs.long().to(device) # Flatten inputs and ensure float type
            embedded_inputs = embedding_layer(inputs)
            autoencoder_optimizer.zero_grad()
            outputs = autoencoder(embedded_inputs)
            # if counter == 0:
            #     print(embedded_inputs.shape)
            #     print(outputs.shape, embedded_inputs.shape)
            #     counter = 1
            loss = criterion(outputs, embedded_inputs)
            loss.backward()
            autoencoder_optimizer.step()

    torch.save(autoencoder.state_dict(), f'{reps_folder_path}/autoencoder.pth')

    model = CNN2D_E2E_AE(vocab_size=vocab_size, num_filters=num_filters, num_class=num_classes, num_channels= num_of_reps, width=max_sentence_length, auto_encoder_path=reps_folder_path).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = lr_wordcnn)
    # for param in model.autoencoder.parameters():
    #     param.requires_grad = False

    config_bert = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_classes)
    model_bert = BertModel.from_pretrained('bert-base-uncased').to(device)
    optimizer_bert = torch.optim.AdamW(model_bert.parameters(), lr=lr_bert)

    model_mlp = MLP(num_filters + config_bert.hidden_size, (num_filters + config_bert.hidden_size) // 2, num_classes ).to(device)
    optimizer_mlp = torch.optim.Adam(model_mlp.parameters(), lr=lr_mlp)

    print(f'start: {rep_name}')

    best_auc = 0.0
    best_preds, best_labels = None, None
    for epoch in range(epochs):
        model.train()
        model_bert.train()
        model_mlp.train()
        train_loss = 0
        train_preds, train_targets = [], []
        for (inputs, labels), inputs_trans in zip(train_loader_3D, train_loader_transformer):
            optimizer.zero_grad()
            optimizer_bert.zero_grad()
            optimizer_mlp.zero_grad()

            inputs, labels = inputs.to(device), labels.to(device)
            inputs_trans, attention_mask_trans = inputs_trans['input_ids'].to(device), inputs_trans['attention_mask'].to(device)

            outputs = model(inputs, output_last_layer=True)
            outputs_trans = model_bert(inputs_trans, attention_mask=attention_mask_trans)[0]
            outputs_bt_cls = outputs_trans[:, 0]

            combined_output = torch.cat((outputs, outputs_bt_cls), dim=1)
            outputs_mlp = model_mlp(combined_output)

            loss = F.cross_entropy(outputs_mlp, labels)
            loss.backward()
            optimizer.step()
            optimizer_bert.step()
            optimizer_mlp.step()

            train_loss += loss.item()
            # _, preds = torch.max(outputs, 1)
            preds = torch.argmax(outputs_mlp, dim=-1)
            train_preds.extend(preds.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_loss /= len(train_loader_3D)  # average loss over the epoch
        train_acc = (np.array(train_preds) == np.array(train_targets)).mean()  # accuracy
        train_auc = roc_auc_score(train_targets, train_preds)  # AUC
        train_recall = recall_score(train_targets, train_preds)  # recall
        train_f1 = f1_score(train_targets, train_preds)  # F1
        train_f2 = fbeta_score(train_targets, train_preds, beta=2)  # F2

        model.eval()
        model_bert.eval()
        model_mlp.eval()

        val_loss = 0
        val_preds, val_targets = [], []
        with torch.no_grad():
            for (inputs, labels), inputs_trans in zip(val_loader_3D, validation_loader_transformer):
                inputs, labels = inputs.to(device), labels.to(device)
                # if val_loss == 0: print(inputs.shape)
                inputs_trans, attention_mask_trans = inputs_trans['input_ids'].to(device), inputs_trans['attention_mask'].to(device)
                outputs = model(inputs, output_last_layer=True)
                outputs_trans = model_bert(inputs_trans, attention_mask=attention_mask_trans)[0]
                outputs_bt_cls = outputs_trans[:, 0]# Use only the [CLS] token representation from BERT and flatten WordCNN output
                combined_output = torch.cat((outputs, outputs_bt_cls), dim=1)
                outputs_mlp = model_mlp(combined_output)
                loss = F.cross_entropy(outputs_mlp, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs_mlp, dim=-1)
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
        val_loss /= len(val_loader_3D)  # average loss over the epoch
        val_acc = (np.array(val_preds) == np.array(val_targets)).mean()  # accuracy
        val_auc = roc_auc_score(val_targets, val_preds)  # AUC
        val_recall = recall_score(val_targets, val_preds)  # recall
        val_f1 = f1_score(val_targets, val_preds)  # F1
        val_f2 = fbeta_score(val_targets, val_preds, beta=2)  # F2

        test_loss = 0
        test_preds, test_targets = [], []
        with torch.no_grad():
            for (inputs, labels), inputs_trans in zip(test_loader_3D, test_loader_transformer):
                inputs, labels = inputs.to(device), labels.to(device)
                inputs_trans, attention_mask_trans = inputs_trans['input_ids'].to(device), inputs_trans['attention_mask'].to(device)

                outputs = model(inputs, output_last_layer=True)
                outputs_trans = model_bert(inputs_trans, attention_mask=attention_mask_trans)[0]
                outputs_bt_cls = outputs_trans[:, 0]# Use only the [CLS] token representation from BERT and flatten WordCNN output
                combined_output = torch.cat((outputs, outputs_bt_cls), dim=1)
                outputs_mlp = model_mlp(combined_output)

                loss = F.cross_entropy(outputs_mlp, labels)
                test_loss += loss.item()
                # _, preds = torch.max(outputs, 1)
                preds = torch.argmax(outputs_mlp, dim=-1)
                test_preds.extend(preds.cpu().numpy())
                test_targets.extend(labels.cpu().numpy())

        test_loss /= len(test_loader_3D)  # average loss over the epoch
        test_acc = (np.array(test_preds) == np.array(test_targets)).mean()  # accuracy
        test_auc = roc_auc_score(test_targets, test_preds)  # AUC
        test_recall = recall_score(test_targets, test_preds)  # recall
        test_f1 = f1_score(test_targets, test_preds)  # F1
        test_f2 = fbeta_score(test_targets, test_preds, beta=2)  # F2

        if test_auc > best_auc:
            best_auc = test_auc
            best_preds = test_preds
            best_labels = test_targets
            stats = f'Stats: \n' \
            f'Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, F2: {train_f2:.4f}\n' \
            f'Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}, F2: {val_f2:.4f}\n' \
            f'Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, F2: {test_f2:.4f}\n\n'
            print(stats)
        else:
            continue

    # stats = f'Stats: \n' \
    #         f'Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, F2: {train_f2:.4f}\n' \
    #         f'Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}, F2: {val_f2:.4f}\n' \
    #         f'Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, F2: {test_f2:.4f}\n\n'
    # print(stats)

### Remove the autoencoder file
os.remove(f'{reps_folder_path}/autoencoder.pth')


In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available")
    device = torch.device("cpu")

# Print GPU details
print(torch.cuda.get_device_name(0))


Bert with embeddings --> Classification

Bert with textagon --> Classification